<a href="https://colab.research.google.com/github/oskargirardin/CRP_GDPR_datasets/blob/TS/time_series_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/rocm5.2
#! pip install sdv --quiet
!pip uninstall torch -y

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118


In [22]:
!pip install torch==1.13.1
! pip install sdv --quiet

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 102.6/102.6 MB 168.0 MB/s eta 0:00:01
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(

In [9]:
local = False

In [21]:
import torch
torch.cuda.is_available()
torch.__version__

'1.13.1+cu117'

In [11]:
import sys

import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
import collections as coll

from sdv.sequential.par import PARSynthesizer
from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata

In [12]:
if local:
    sys.path.append('../../src/')
    from similarity_check.SimilarityCheck import *
    from synthetic_data_generation.generator import *
    from utils import *
    # Import libraries
    from privacy_check.privacy_check import PrivacyCheck

In [14]:
datasets = load_csvs(folder_name='./data/') if local else load_csvs(folder_name='.')

In [15]:
df = datasets["energy_dataset"]
df = df[["time", "generation biomass", "generation fossil brown coal/lignite", "generation fossil gas"]]
df["id"] = df.index
df["time"] = df["time"].apply(lambda x: x[:-6])

<ipython-input-15-4f514de26409>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df.index
<ipython-input-15-4f514de26409>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = df["time"].apply(lambda x: x[:-6])


In [16]:
df

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,id
0,2015-01-01 00:00:00,447.0,329.0,4844.0,0
1,2015-01-01 01:00:00,449.0,328.0,5196.0,1
2,2015-01-01 02:00:00,448.0,323.0,4857.0,2
3,2015-01-01 03:00:00,438.0,254.0,4314.0,3
4,2015-01-01 04:00:00,428.0,187.0,4130.0,4
...,...,...,...,...,...
35059,2018-12-31 19:00:00,297.0,0.0,7634.0,35059
35060,2018-12-31 20:00:00,296.0,0.0,7241.0,35060
35061,2018-12-31 21:00:00,292.0,0.0,7025.0,35061
35062,2018-12-31 22:00:00,293.0,0.0,6562.0,35062


In [17]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)
metadata.update_column(
    "id",
    sdtype='id',
)
metadata.set_sequence_key("id")
metadata.update_column(
    column_name='time',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S')
metadata.set_sequence_index(column_name='time')

In [18]:
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "sequence_key": "id",
    "sequence_index": "time",
    "columns": {
        "time": {
            "sdtype": "datetime",
            "datetime_format": "%Y-%m-%d %H:%M:%S"
        },
        "generation biomass": {
            "sdtype": "numerical"
        },
        "generation fossil brown coal/lignite": {
            "sdtype": "numerical"
        },
        "generation fossil gas": {
            "sdtype": "numerical"
        },
        "id": {
            "sdtype": "id"
        }
    }
}

# Possible methods
For the generation of synthetic datasets with time series data, we have several methods at our disposal:
- PARSynthesizer (```sdv```): Probabilistic AutoRegressive model that allows learning multi-type, multivariate timeseries data and later on generate new synthetic data that has the same format and properties as the learned one. (https://arxiv.org/pdf/2207.14406.pdf)
- DGAN (```gretel-synthetics```)

In [19]:
# Step 1: Create the synthesizer
synthesizer = PARSynthesizer(metadata, verbose = True)

# Step 2: Train the synthesizer
synthesizer.fit(df)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

ContextualVersionConflict: ignored